The following codes  
https://towardsdatascience.com/selenium-tweepy-to-scrap-tweets-from-tweeter-and-analysing-sentiments-1804db3478ac
https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/  
https://medium.freecodecamp.org/how-to-make-your-own-sentiment-analyzer-using-python-and-googles-natural-language-api-9e91e1c493e  
https://www.programcreek.com/python/example/76301/tweepy.Cursor  
https://cmry.github.io/notes/twitter-python


In [1]:
# !pip3 install tweepy

In [1]:
import os,sys,inspect
import query_terms
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
import twitterkeys
import tweepy
import pandas as pd 
from tweepy import OAuthHandler

In [2]:
consumer_key = twitterkeys.CONSUMER_KEY
consumer_secret   = twitterkeys.CONSUMER_SECRET
access_token   = twitterkeys.ACCESS_TOKEN
access_token_secret   = twitterkeys.ACCESS_TOKEN_SECRET

In [3]:
angry_query_a = query_terms.ANGRY_QUERY_a
angry_query_b = query_terms.ANGRY_QUERY_b
angry_query_c = query_terms.ANGRY_QUERY_c
fear_query_a = query_terms.FEAR_QUERY_a
fear_query_b = query_terms.FEAR_QUERY_b
fear_query_c = query_terms.FEAR_QUERY_c
fear_query_d = query_terms.FEAR_QUERY_d
sad_query_a = query_terms.SAD_QUERY_a
sad_query_b = query_terms.SAD_QUERY_b
sad_query_c = query_terms.SAD_QUERY_c
sad_query_d = query_terms.SAD_QUERY_d

In [4]:
def authentication(cons_key, cons_secret, acc_token, acc_secret):
    auth = tweepy.OAuthHandler(cons_key, cons_secret)
    auth.set_access_token(acc_token, acc_secret)
    api = tweepy.API(auth)
    return api

In [47]:
from datetime import datetime, timedelta
today_datetime = datetime.today().now()
yesterday_datetime = today_datetime - timedelta(days=1)
today_date = today_datetime.strftime('%Y-%m-%d')
yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
print(yesterday_datetime.strftime('%Y-%m-%d-%H-%M'))

2019-05-28-01-53


## Test scrapping mechanism

In [41]:
def search_tweets_no_date(keyword, total_tweets):
    today_datetime = datetime.today().now()
    yesterday_datetime = today_datetime - timedelta(days=1)
    today_date = today_datetime.strftime('%Y-%m-%d')
    yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
    api = authentication(consumer_key,consumer_secret,access_token,access_token_secret)
    search_result = tweepy.Cursor(api.search, 
                                  q=keyword, 
                                  since=yesterday_date, 
                                  result_type='recent', 
                                  tweet_mode='extended',
                                  lang='en').items(total_tweets)
    
    # Add tweets in this format
    listOfTweets = []
    
    print(yesterday_date, yesterday_datetime)
    for tweet in search_result:
        # Add tweets in this format
        if tweet.user.followers_count < 100:
            continue
        else:
            dict_ = {'Screen Name': tweet.user.screen_name,
                    'User Name': tweet.user.name,
                    'User Location': tweet.user.location,       
                    'User Likes':tweet.user.favourites_count,
                    'User Followers':tweet.user.followers_count,
                    'User Following':tweet.user.friends_count,
                    'User Total Tweets':tweet.user.statuses_count, 
                    'User Created Time': tweet.user.created_at,
                    'User Description': tweet.user.description,

                    'Tweet Text': tweet.full_text,                                  
                    'Tweet Created Time': tweet.created_at,
                    'Tweet Likes': tweet.favorite_count,
                    'Tweet Retweet Count':tweet.retweet_count,                 
                    'Tweet Coordinates': tweet.coordinates,                
                    'Tweet As Retweet': tweet.retweeted,
                    'Tweet Platform Type': tweet.source,
                    'Tweet Language': tweet.lang,


                    'Tweet Favorited': tweet.favorited,
                    'Tweet In reply to': tweet.in_reply_to_status_id_str,
                    'Tweet id': tweet.id_str,
                    'Tweet URL': 'https://twitter.com/i/web/status/'+ tweet.id_str
                    }
            listOfTweets.append(dict_)   
    return pd.DataFrame(listOfTweets)

In [42]:
today_datetime = datetime.today().now()
yesterday_datetime = today_datetime - timedelta(days=1)
today_date = today_datetime.strftime('%Y-%m-%d')
yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
angry_a = search_tweets_no_date(angry_query_a,100)
angry_a

2019-05-27 2019-05-27 07:02:17.523003


,Screen Name,Tweet As Retweet,Tweet Coordinates,Tweet Created Time,Tweet Favorited,Tweet In reply to,Tweet Language,Tweet Likes,Tweet Platform Type,Tweet Retweet Count,...,Tweet URL,Tweet id,User Created Time,User Description,User Followers,User Following,User Likes,User Location,User Name,User Total Tweets
0,Shulab,False,None,2019-05-28 07:02:00,False,None,en,0,IFTTT,0,...,https://twitter.com/i/web/status/1133267158622...,1133267158622978048,2009-02-03 08:20:51,#Adventures #FreeSpirit #clouds #music #Modern...,1425,1090,9505,Oman,Yusra Salim | يُسرى بِنْت سالم,137570
1,Daily_Star,False,None,2019-05-28 07:02:00,False,None,en,0,TweetDeck,0,...,https://twitter.com/i/web/status/1133267158291...,1133267158291533824,2009-02-09 15:45:17,Your Official Daily Star Twitter Page!,185470,3631,824,London,Daily Star,443246
2,emmy_rudolph,False,None,2019-05-28 07:01:56,False,None,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1133267143846...,1133267143846387713,2014-02-18 00:22:53,Have you tried cheese? Syracuse MT '21 🍊,612,1160,14431,seattle/syracuse,emily rudolph :),3277
3,UNDISPUTED_SAM1,False,None,2019-05-28 07:01:56,False,1133260354467893248,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1133267142789...,1133267142789537793,2015-10-01 12:05:58,"The Scripture🙏, The Future👍 and My Pictures 💝😎",752,1846,1628,UAE,UNdIspuTeD,873
4,beswitched1,False,None,2019-05-28 07:01:55,False,1133265543807623168,en,0,Twitter Web App,0,...,https://twitter.com/i/web/status/1133267139283...,1133267139283099648,2016-08-24 15:50:29,Author of bespoke spanking stories. Lover of r...,1218,252,1628,West Yorkshire,Adrian,5398
5,craigyhunn,False,None,2019-05-28 07:01:55,False,1133173786688204800,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1133267138029...,1133267138029002752,2013-05-29 16:20:19,38. Solicitor. NUFC victim.,262,493,12214,North Shields,Craig Hunn,2910
6,DeShaunsMrk,False,None,2019-05-28 07:01:51,False,None,en,0,IFTTT,0,...,https://twitter.com/i/web/status/1133267123323...,1133267123323756545,2009-05-26 14:53:14,#BrandAuthority | #Branding | #Consulting | #M...,3340,2966,344,"Cleveland, Ohio",DeShaunsMrk™,182302
7,mr_ed_wilson,False,None,2019-05-28 07:01:49,False,1133117486826754050,en,0,Twitter Web Client,0,...,https://twitter.com/i/web/status/1133267114788...,1133267114788364288,2015-03-05 20:20:03,Researcher. Trequartista. Late 20s/early-mid-l...,283,473,5473,,Serenity Now 🔸,2698
8,PrettyBoyNikoo,False,None,2019-05-28 07:01:49,False,None,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1133267111407...,1133267111407628289,2016-01-05 18:05:10,,374,608,15524,,nick at night,1386
9,utxpicpain,False,None,2019-05-28 07:01:47,False,None,en,0,SumAll,0,...,https://twitter.com/i/web/status/1133267104512...,1133267104512258048,2016-01-17 18:13:16,Who knew evil girls have the prettiest face?\n...,477,265,8956,Acariciando un gatete,ᴊᴏʟʏɴᴇ 🍑,5086


### Test saving and loading from csv and pandas

In [50]:
# today_datetime = datetime.today().now()
# yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
# filename = '/data/SuperMod/TwitterData/anger' + yesterday_date + '.csv'

In [51]:

# angry_b.to_csv(path_or_buf =filename , index=False)

In [52]:
# pd = pd.read_csv(filename)
# pd

,Screen Name,Tweet As Retweet,Tweet Coordinates,Tweet Created Time,Tweet Favorited,Tweet In reply to,Tweet Language,Tweet Likes,Tweet Platform Type,Tweet Retweet Count,...,Tweet URL,Tweet id,User Created Time,User Description,User Followers,User Following,User Likes,User Location,User Name,User Total Tweets
0,mrithejester,False,NaN,2019-05-27 08:24:09,False,NaN,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132925446134...,1132925446134358016,2018-01-13 12:22:00,I exist. No religion. No DMs. 🚫,1993,441,25413,K2-229 b,Paul,24730
1,satya_AmitSingh,False,NaN,2019-05-27 08:24:07,False,1.132916e+18,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132925437166...,1132925437166800896,2011-12-22 04:27:02,"Nationalist 🇮🇳Proud Hindu🧡& Entrepreneur,CEO w...",259,168,10482,"New Delhi, Delhi",Satyajit Amit Singh,4019
2,DumbxssHooker,False,NaN,2019-05-27 08:24:05,False,NaN,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1132925429814...,1132925429814366208,2019-02-16 23:27:37,⠀⠀⠀❪ ·𖥸· ❫ ｜ ➻ 〝 ᴀʀᴇ ʏᴏᴜ ᴍᴀᴅ ᴏʀ ᴀʀᴇ ʏᴏᴜ ꜱᴛᴀʀɪɴ...,195,142,293,#𝔾𝕚𝕟𝕘𝕖𝕣ℙ𝕒𝕤𝕥𝕪 | #Fᴀᴋᴇ,ㅤ ㅤ𝐑. 𝐋𝐞𝐬𝐭𝐫𝐚𝐧𝐠𝐞,146


In [15]:
# api.rate_limit_status()['resources']

## put everything in functions

In [54]:
# rewrite this function again for scrapping

def search_tweets(keyword, total_tweets, last_time, cur_time):

    api = authentication(consumer_key,consumer_secret,access_token,access_token_secret)
    search_result = tweepy.Cursor(api.search, 
                                  q=keyword, 
                                  since = last_time, 
#                                   until= cur_time,
                                  result_type='recent', 
                                  tweet_mode='extended',
                                  lang='en').items(total_tweets)
    
    # Add tweets in this format
    listOfTweets = []
    
    for tweet in search_result:
        # Add tweets in this format
        if tweet.user.followers_count < 100:
            continue
        else:
            dict_ = {'Screen Name': tweet.user.screen_name,
                    'User Name': tweet.user.name,
                    'User Location': tweet.user.location,       
                    'User Likes':tweet.user.favourites_count,
                    'User Followers':tweet.user.followers_count,
                    'User Following':tweet.user.friends_count,
                    'User Total Tweets':tweet.user.statuses_count, 
                    'User Created Time': tweet.user.created_at,
                    'User Description': tweet.user.description,

                    'Tweet Text': tweet.full_text,                                  
                    'Tweet Created Time': tweet.created_at,
                    'Tweet Likes': tweet.favorite_count,
                    'Tweet Retweet Count':tweet.retweet_count,                 
                    'Tweet Coordinates': tweet.coordinates,                
                    'Tweet As Retweet': tweet.retweeted,
                    'Tweet Platform Type': tweet.source,
                    'Tweet Language': tweet.lang,


                    'Tweet Favorited': tweet.favorited,
                    'Tweet In reply to': tweet.in_reply_to_status_id_str,
                    'Tweet id': tweet.id_str,
                    'Tweet URL': 'https://twitter.com/i/web/status/'+ tweet.id_str
                    }
            listOfTweets.append(dict_)   
    return pd.DataFrame(listOfTweets)

def scrape_by_date(list_of_sizes, days_ago, skip_fear):
    """
    This function scrapes Twitter for a certain number of messages for different query words
    and scrapes for a specific day from today
    list_of_sizes: list of number of tweets to scrape by keyword lists. 
                   Not all queries return enough words, if don't specify will run into issues
    days_ago: an integer, specify how many days ago do we want to scrape, 
             today is 1/10/19, if I put 2, then I will be scrapping from 1/7/19 to 1/8/19
    skip_fear: do not scrape for fear if True, on certain days there are simply not enough to scrape
    
    For future scrapes, do it only for single day, so do not include 'until' in search API
    """
    
    today_datetime = datetime.today().now()
    
    end_datetime = today_datetime - timedelta(days = days_ago)
    start_datetime = end_datetime - timedelta(days = 1)
    
    start_date = start_datetime.strftime('%Y-%m-%d')
    end_date = end_datetime.strftime('%Y-%m-%d')
    print_date = end_datetime.strftime('%Y-%m-%d-%H-%M')
    
    print("scrapping from date", start_date, "to date", end_date, "at time", start_datetime)
    
    
    angry_a = search_tweets(angry_query_a, list_of_sizes[0], start_date, end_date)
    angry_b = search_tweets(angry_query_b, list_of_sizes[1], start_date, end_date)
    angry_c = search_tweets(angry_query_c, list_of_sizes[2], start_date, end_date)
    
    sad_a = search_tweets(sad_query_a, list_of_sizes[7], start_date, end_date)
    sad_b = search_tweets(sad_query_b, list_of_sizes[8], start_date, end_date)
    sad_c = search_tweets(sad_query_c, list_of_sizes[9], start_date, end_date)       
    sad_d = search_tweets(sad_query_d, list_of_sizes[10], start_date, end_date)      

    angry = angry_a.append([angry_b,angry_c])
    sad = sad_a.append([sad_b, sad_c, sad_d])   
            
    base_path = '/data/SuperMod/TwitterData/'

    filenameanger = base_path + 'anger/anger' + print_date + '.csv'    
    filenamesad = base_path + 'sad/sad' + print_date + '.csv'  
    
    if len(angry) != 0:
        angry.to_csv(path_or_buf =filenameanger , index=False)
    if len(sad) != 0:
        sad.to_csv(path_or_buf =filenamesad , index=False)

    if not skip_fear:
        
        fear_a = search_tweets(fear_query_a, list_of_sizes[3], start_date, end_date)
        fear_b = search_tweets(fear_query_b, list_of_sizes[4], start_date, end_date)
        fear_c = search_tweets(fear_query_c, list_of_sizes[5], start_date, end_date) 
        fear_d = search_tweets(fear_query_d, list_of_sizes[6], start_date, end_date)   
    
        fear = fear_a.append([fear_b, fear_c, fear_d])
        
        print("angry:" ,len(angry), "fear:", len(fear), "sad:", len(sad))
    
        filenamefear = base_path + 'fear/fear' + print_date + '.csv'   
        
        if len(fear) != 0: 
            fear.to_csv(path_or_buf =filenamefear , index=False)

    else: 
        print("angry:" ,len(angry),  "sad:", len(sad))   


## On some days there may not be that many tweets with emotional indicators
## Insterestingly, by specifying date range, there are fewer results
## Just have to scrape every day

And the results returned are Tweets posted wihtin a few min. So we can choose different time period to scrape


In [49]:
##
sizes_lar = [100, 100, 100, 5, 10, 10, 10, 10, 100, 100, 100]
sizes_med = [10, 50, 50, 5, 10, 10, 10, 10, 50, 50, 50]
sizes_sma = [10, 10, 10, 5, 10, 10, 10, 10, 10, 10, 10]



In [50]:
days = 0

In [59]:


try:
    print("scraping large size with fear")
    scrape_by_date(sizes_lar, days, False)
except:
    try:
        print("can't scrape large size with fear")
        print("scraping medium size with fear")        
        scrape_by_date(sizes_med, days, False)
    except:
        print("can't scrape medium size with fear")
        print("scraping large size without fear")
        try: 
            scrape_by_date(sizes_lar, days, True)
        except:
            print("can't scrape large size without fear")
            print("scraping medium size without fear")
            try:
                scrape_by_date(sizes_med, days, True)
            except:
                print("can't scrape medium size without fear")
                print("scraping small size without fear")       
                scrape_by_date(sizes_sma, days, True)



scraping large size with fear
scrapping from date 2019-05-28 to date 2019-05-29 at time 2019-05-28 04:22:32.919247
angry: 227 fear: 29 sad: 223


In [ ]:
# today_datetime = datetime.today().now()
# today_datetime = today_datetime - timedelta(days=1)
# today_date = today_datetime.strftime('%Y-%m-%d')
# yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
# print(today_date)

In [52]:
# today_datetime = datetime.today().now()- timedelta(days=3)
# yesterday_datetime = today_datetime - timedelta(days=1)
# today_date = today_datetime.strftime('%Y-%m-%d')
# yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
# print("scrapping from date", yesterday_date, "to date", today_date)
# angry_b = search_tweets(angry_query_a,10,yesterday_date,today_date)
# angry_b

scrapping from date 2019-05-24 to date 2019-05-25


,Screen Name,Tweet As Retweet,Tweet Coordinates,Tweet Created Time,Tweet Favorited,Tweet In reply to,Tweet Language,Tweet Likes,Tweet Platform Type,Tweet Retweet Count,...,Tweet URL,Tweet id,User Created Time,User Description,User Followers,User Following,User Likes,User Location,User Name,User Total Tweets
0,Kendalsnowflake,False,None,2019-05-24 23:59:59,False,1132071306592174085,en,3,Twitter for Android,0,...,https://twitter.com/i/web/status/1132073793281...,1132073793281642497,2010-05-22 18:18:37,Director. Cinematographer. Seahawks fan. Socia...,181,349,8513,"Los Angeles, CA",Kendall Christianson,8864
1,GButtafly,False,None,2019-05-24 23:59:58,False,None,en,4,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132073788160...,1132073788160565254,2018-02-16 05:42:50,Findom Goddess. ISO pay 🐷 💰 💵 I’ll take all of...,165,71,8,,🦋GoddessButtafly 🦋,248
2,tgs102002,False,None,2019-05-24 23:59:58,False,1132062674257211392,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1132073787170...,1132073787170729986,2011-03-27 11:27:51,"yup, these are the things I think about.",114,259,2372,Chicago,t.g.s,2659
3,tiffanytrent,False,None,2019-05-24 23:59:57,False,1132072940059734016,en,1,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132073783358...,1132073783358119936,2009-03-09 22:25:22,Author of science fiction and fantasy for adul...,3268,1922,28179,"Here, there, and everywhere",Tiffany Trent,43057
4,ashleyecrosby,False,None,2019-05-24 23:59:57,False,None,en,3,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132073781646...,1132073781646766081,2009-11-19 18:21:05,Strength & Nutrition Coach | Always at a baseb...,976,658,7905,"Jupiter, FL","Ashley Wittgren, MS CSCS",23337
5,Reematefarag,False,None,2019-05-24 23:59:54,False,None,en,1,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132073772654...,1132073772654239745,2013-02-06 13:21:50,مش هعمل فولو باك لكل الناس لسبب ما، هندسة .. ا...,2319,288,72275,"Misr Al-Gadeeda,Cairo,Egypt","As a matter of fact, I",61688
6,saba_dee,False,None,2019-05-24 23:59:51,False,1132073389550657538,en,1,Twitter for Android,0,...,https://twitter.com/i/web/status/1132073759530...,1132073759530278913,2014-02-26 02:31:05,Eagle 🦅|| Sociologist || Introvert || God love...,386,373,6169,"Ibadan, Nigeria",King Saba,3402
7,YolandHowden,False,None,2019-05-24 23:59:46,False,1131851246317518848,en,1,Twitter for Android,0,...,https://twitter.com/i/web/status/1132073736667...,1132073736667111424,2009-10-26 23:36:14,If you're happy with your floorboards no-one c...,203,275,1695,Staffordshire,Yoland Howden,1527


### read in results to check

In [55]:
filename = '/data/SuperMod/TwitterData/anger/2019-05-28-04-05.csv'

In [56]:
test_load = pd.read_csv(filename)
test_load.head()


,Screen Name,Tweet As Retweet,Tweet Coordinates,Tweet Created Time,Tweet Favorited,Tweet In reply to,Tweet Language,Tweet Likes,Tweet Platform Type,Tweet Retweet Count,...,Tweet URL,Tweet id,User Created Time,User Description,User Followers,User Following,User Likes,User Location,User Name,User Total Tweets
0,mohammedhersi,False,NaN,2019-05-29 04:05:08,False,1.132655e+18,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1133585038484...,1133585038484869120,2010-09-27 08:31:33,#OPTIMIST . Chairman. Kenya Tourism Federation...,251664,14194,16222,"Mombasa, Kenya",Mohammed Hersi,40243
1,ProperOpinion,False,NaN,2019-05-29 04:05:08,False,1.133584e+18,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1133585036190...,1133585036190593025,2017-08-20 00:08:34,Consider the possibility that I'm joking. Also...,17177,2338,330339,On a steel horse with Bon Jovi,prop op,97642
2,Shakirah_Chante,False,NaN,2019-05-29 04:05:06,False,1.133531e+18,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1133585030050...,1133585030050009089,2016-06-04 00:48:10,"I’m Not Broke, I’m Pre-rich ...",229,402,13620,"Los Angeles, CA",Shakirah2x,3175
3,AVSRathore1,False,NaN,2019-05-29 04:05:06,False,1.133583e+18,en,1,Twitter for Android,0,...,https://twitter.com/i/web/status/1133585028300...,1133585028300935169,2015-12-04 01:27:13,BJP - \n🇮🇳.. I WORK ONLY ON FACTS. ..........,162,93,4740,"Delhi , india",Ajay Rathore 2.0🗨,5508
4,Faygoluvers,False,NaN,2019-05-29 04:05:05,False,NaN,en,0,Tweet Posts (Nextscripts),0,...,https://twitter.com/i/web/status/1133585024517...,1133585024517722112,2010-04-29 16:08:05,http://t.co/JbgpdwmBIO is your #1 source for n...,34020,599,3012,"Dallas, TX",Faygoluvers.net,28059


In [58]:
test_load.iloc[-1,]


Screen Name                                                    Tonyg6200
Tweet As Retweet                                                   False
Tweet Coordinates                                                    NaN
Tweet Created Time                                   2019-05-29 04:04:37
Tweet Favorited                                                    False
Tweet In reply to                                                    NaN
Tweet Language                                                        en
Tweet Likes                                                            0
Tweet Platform Type                                   Twitter for iPhone
Tweet Retweet Count                                                    0
Tweet Text                              Damn karma really on my ass rn 😤
Tweet URL              https://twitter.com/i/web/status/1133584905437...
Tweet id                                             1133584905437294592
User Created Time                                  

In [ ]:
for i in test_load['Tweet Text']:
    print(i)

In [65]:
filename = '/data/SuperMod/TwitterData/sad/sad2019-05-29-01-57.csv'
test_load = pd.read_csv(filename)
test_load.head()

,Screen Name,Tweet As Retweet,Tweet Coordinates,Tweet Created Time,Tweet Favorited,Tweet In reply to,Tweet Language,Tweet Likes,Tweet Platform Type,Tweet Retweet Count,...,Tweet URL,Tweet id,User Created Time,User Description,User Followers,User Following,User Likes,User Location,User Name,User Total Tweets
0,bobjones2244,False,NaN,2019-05-29 01:57:44,False,1.133552e+18,en,0,Twitter Web Client,0,...,https://twitter.com/i/web/status/1133552974251...,1133552974251405313,2016-04-01 00:46:08,"🇺🇸🇳🇴 Atlanticist, Protestant, Moderate▪️ 📘 Gra...",309,154,26306,The Swamp. Nipple & Button for vacation.,Gandalf🐺🇺🇸🇬🇧🇦🇺🐺,13662
1,SexyInMy60s,False,NaN,2019-05-29 01:57:43,False,NaN,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1133552973521...,1133552973521534978,2009-04-22 01:48:52,"Age Reversal Expert, Author, Motivational Spea...",883,2044,1716,Austin,Joan Bunney,9737
2,dekofler,False,NaN,2019-05-29 01:57:43,False,1.133552e+18,en,0,Twitter Web App,0,...,https://twitter.com/i/web/status/1133552970912...,1133552970912583681,2017-01-25 20:28:03,RN. Outdoor enthusiast. Lover of all things ca...,204,875,13079,"San Francisco, CA",Ice•Climbing•RN 🌊🌊🌊🌊🌊,3075
3,xvllva,False,NaN,2019-05-29 01:57:41,False,NaN,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1133552963702...,1133552963702665217,2016-04-12 17:11:19,#capricorn,635,67,368,NaN,A,254
4,Charlie__Neal,False,NaN,2019-05-29 01:57:38,False,1.133551e+18,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1133552950792...,1133552950792609792,2014-06-20 00:09:51,19 #Titans #Nuggets Happily blocked by @jalenr...,637,575,33703,NaN,Charlie (check mark),20762


In [66]:
for i, j in zip(test_load['Tweet Text'],test_load['Tweet URL']) :
    print(i)
    print(j)

@wokyleeks @LeslieMPozsonyi @SueUvino @ddp @PhlItalian @JupiterDi1112 @DisChimera @Imwastntym @2manydeserts @LFredenhall @thespybrief @AZChiweenerPop @Pooterbugsmom @hiemakene @JulieO_TO @SylviaDonoghue3 @Dark_Falcon7 @soychicka that's pretty fuqqing scary and bad if true. there's not much that's worse ( more powerful) than raping kids.
https://twitter.com/i/web/status/1133552974251405313
Graham has no courage or character. He's a dismal shell of a man. Shame on his despicable behavior. https://t.co/puL4G0Zrl6
https://twitter.com/i/web/status/1133552973521534978
@measi @hazeleyesgirl2 I think that is the crux of it-people want a baby like it is a 'thing,' something they are owed or can go to a store (if wealthy) and purchase. The 'why wasn't I good enough/why wasn't I kept' nagged at my friend for a long time &amp; resulted in his eating disorder/depression :-(
https://twitter.com/i/web/status/1133552970912583681
Real depression is when you stop loving the things you love
https://twitt